<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук НИУ ВШЭ Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее  задание № 9</center>
## <center>Vowpal Wabbit в задаче идентификации пользователей по посещенным веб-сайтам. Решение</center>

[Веб-форма](https://goo.gl/forms/rnUMG9jXTi2qJ6mk2) для ответов.

In [84]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

**Считайте данные [соревнования](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data) в DataFrame train_df и test_df (обучающая и тестовая выборки).**

In [3]:
train_df = pd.read_csv('../../data/websites_train_sessions.csv', index_col='session_id')
test_df = pd.read_csv('../../data/websites_test_sessions.csv', index_col='session_id')

**Пока в качестве признаков используем только посещенные сайты.**

**Выделите в отдельный вектор *y* ответы на обучающей выборке.**

**Выделите сайты и переведите данные в текстовый формат, который можно подать на вход `CountVectorizer`.**

Пример: первая сессия состоит из сайтов 8, 11, 82, 68, 8, 8393, 8403, 932, 3260, 8. Надо это преобразовать в строку "8 11 82 68 8 8393 8403 932 3260 8".
Кстати, это можно сделать еще на этапе чтения данных из файла.




In [40]:
sites = ['site' + str(i) for i in range(1,11)]
train_df_sites = train_df[sites].fillna(0).astype('int')
test_df_sites = test_df[sites].fillna(0).astype('int')
y = train_df['user_id']

In [8]:
train_df_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,8,11,82,68,8,8393,8403,932,3260,8
2,111,78,151,111,1401,151,1375,38,1401,97
3,11,3187,132,496,1969,504,3187,82,3191,3184
4,668,1965,598,1965,284,668,38,4451,4537,11
5,1943,1943,1943,1943,1943,1952,1952,1943,1943,1943


In [41]:
train_sessions_text = train_df_sites.apply(lambda row: 
                      ' '.join([str(row[i]) for i in range(10)]),
                      axis=1)
test_sessions_text = test_df_sites.apply(lambda row: 
                      ' '.join([str(row[i]) for i in range(10)]),
                      axis=1)

In [42]:
count_vectorizer = CountVectorizer()

In [43]:
X_train_sparse = count_vectorizer.fit_transform(train_sessions_text)
X_test_sparse = count_vectorizer.transform(test_sessions_text)

In [44]:
X_train_sparse.shape, X_test_sparse.shape

((95319, 21523), (41177, 21523))

**<font color='red'> Вопрос 1:</font> Укажите число признаков (столбцов) в разреженных матрицах X_train_sparse и X_test_sparse.**

**<font color='red'> Ответ:</font> 21523**

**Vowpal Wabbit любит, чтоб метки классов были распределены от 1 до K, где K – число классов в задаче классификации (в нашем случае – 550). Поэтому придется применить LabelEncoder, да еще и +1 потом добавить (LabelEncoder переводит метки в диапозон от 0 до K-1).**

In [47]:
class_encoder = LabelEncoder().fit(y)

In [48]:
y_for_vw = class_encoder.transform(y) + 1

**Выделим обучающую и оставленную части исходной обучающей выборки.**

In [51]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_sparse, y_for_vw, test_size=0.3, 
                                                     random_state=7, stratify=y_for_vw)

**Реализуйте функцию, переводящую разреженную матрицу в формат Vowpal Wabbit.**

Вход:
 - X_sparse – разреженная матрица SciPy.sparse.csr_matrix
 - y (необяз.) - вектор ответов. Необязателен, поскольку тестовую матрицу будем обрабатывать этой же функцией
 - out_file – путь к файлу .vw, в который будет произведена запись
 
Детали:
- можно делать по-разному, но скорее всего поможет метод *nonzero* разреженной матрицы
- в тестовой выборке на месте меток целевого класса можно писать произвольные, допустим, 1

In [52]:
def sparse_matrix_to_vw(X_sparse, y=None, out_file='tmp.vw'):
    with open(out_file, 'w') as out_file:
        for i in range(X_sparse.shape[0]):
            data, ind = X_sparse[i,:].data, X_sparse[i,:].indices + 1
            if sum(data) == len(data):
                
                freqs_series = pd.Series(ind).value_counts()
                sites_freqs = zip(freqs_series.index, freqs_series.values)
            else:
                sites_freqs = zip(ind, data)
                
            s = ''
            for site_id, freq in sites_freqs:
                s += '{}:{} '.format(site_id, freq)
            
                 
            out_file.write('{0} | sites {1}\n'.format(y[i] if y is not None else 1,
                            s.strip()))

**Примените написанную функцию к части обучащей выборки (X_train, y_train), к отложенной выборке (X_valid, y_valid), ко всей обучающей выборке (X_train_sparse, y_for_vw) и ко всей тестовой выборке X_test_sparse.**

In [61]:
%%time
sparse_matrix_to_vw(X_train, y_train, '../../data/web_sessions_train_part.vw')
sparse_matrix_to_vw(X_valid, y_valid, '../../data/web_sessions_valid.vw')
sparse_matrix_to_vw(X_train_sparse, y_for_vw, '../../data/web_sessions_train.vw')
sparse_matrix_to_vw(X_test_sparse, out_file='../../data/web_sessions_test.vw')

CPU times: user 51.5 s, sys: 209 ms, total: 51.7 s
Wall time: 51.8 s


In [62]:
!head -2 ../../data/web_sessions_train_part.vw

373 | sites 1224:1 19082:1 4458:1 14642:1 21191:2 4124:3 8889:1
67 | sites 14642:1 13976:1 16809:2 17197:2 14351:1


In [63]:
!head -2 ../../data/web_sessions_valid.vw

444 | sites 14642:1 13810:2 13999:4 14387:1 17661:1
178 | sites 10446:7 10501:3


In [64]:
!head -2 ../../data/web_sessions_train.vw

298 | sites 19741:1 14044:1 19753:1 1112:1 19526:1 17972:1 20770:1
537 | sites 21191:1 14642:1 4168:1 4458:2 5668:2 19082:1 1224:2


In [65]:
!head -2 ../../data/web_sessions_test.vw

1 | sites 10190:1
1 | sites 11363:5 11399:1 11419:1 13421:1 13976:1


**Обучите модель на выборке kaggle_data/train_part.vw. Укажите, что решается задача классификации с 550 классами (--oaa), сделайте 3 прохода по выборке (--passes). Также укажите значение параметра b=26. Это число бит, используемых для хэширования, в данном случае нужно больше, чем 18 по умолчанию. Остальные параметры пока не меняйте, далее уже в свободном режиме соревнования можете попробовать другии функции потерь.**

In [71]:
PATH_TO_DATA = '../../data'
train_part_vw = os.path.join(PATH_TO_DATA, 'web_sessions_train_part.vw')
valid_vw = os.path.join(PATH_TO_DATA, 'web_sessions_valid.vw')
train_vw = os.path.join(PATH_TO_DATA, 'web_sessions_train.vw')
test_vw = os.path.join(PATH_TO_DATA, 'web_sessions_test.vw')
model = os.path.join(PATH_TO_DATA, 'vw_model.vw')
pred = os.path.join(PATH_TO_DATA, 'vw_pred.csv')

In [73]:
%%time
!vw --oaa 550 $train_part_vw -f $model -b 26 -c -k --random_seed 7 --quiet  --passes=3

CPU times: user 212 ms, sys: 83.4 ms, total: 296 ms
Wall time: 16.5 s


**Запишите прогнозы на выборке *web_sessions_valid.vw* в *vw_pred.csv*.**

In [74]:
%%time
!vw -t -i $model $valid_vw -p $pred --random_seed 7 --quiet

CPU times: user 9.37 ms, sys: 13.3 ms, total: 22.7 ms
Wall time: 628 ms


**Считайте прогнозы *vw_pred.csv*  из файла и посмотрите на долю правильных ответов на отложенной части.**

In [77]:
vw_valid_pred = pd.read_csv(pred, 
                            header=None).values.flatten()
accuracy_score(y_valid, vw_valid_pred)

0.32864736326758986

**<font color='red'> Вопрос 2:</font> Какой получается доля правильных ответов на отложенной выборке?** 

**<font color='red'> Ответ:</font> 0.329**

**Обучите модель с теми же параметрами на всей обучающей выборке – *web_sessions_train.vw*.**

In [78]:
%%time
!vw --oaa 550 $train_vw -f $model \
-b 26 -c -k  --random_seed 7 --quiet --passes=3

CPU times: user 266 ms, sys: 100 ms, total: 366 ms
Wall time: 20.2 s


**Сделайте прогноз для тестовой выборки.**

In [79]:
%%time
!vw -t -i $model $test_vw -p $pred --random_seed 7 --quiet

CPU times: user 11.5 ms, sys: 13 ms, total: 24.5 ms
Wall time: 767 ms


**Запишите прогноз в файл, примените обратное преобразование меток (был LabelEncoder и потом +1 в меткам) и отправьте решение на Kaggle в [соревновании](https://inclass.kaggle.com/c/identify-me-if-you-can-yandex-mipt/data).**

In [80]:
def write_to_submission_file(predicted_labels, out_file,
                             target='user_id', index_label="session_id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [81]:
vw_pred = pd.read_csv(pred, header=None)

In [82]:
vw_subm = class_encoder.inverse_transform((vw_pred.values.flatten() - 1))

In [85]:
write_to_submission_file(vw_subm, 'vw_submision.csv')

**<font color='red'> Вопрос 3:</font> Какой получается доля правильных ответов на публичной части рейтинга Kaggle?**

**<font color='red'> Ответ:</font> 0.372**